In [1]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm

In [2]:
class CoffeeModel(nn.Module):
    def __init__(self,in_features):
        super(CoffeeModel,self).__init__()
        self.layer1 = nn.Linear(in_features,3)
        self.layer2 = nn.Linear(3,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x_in):
        a1 = self.sigmoid(self.layer1(x_in))
        a2 = self.sigmoid(self.layer2(a1))
        return a2

    

In [3]:
def train(x,y,epochs,model):
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.03)
    p_bar = tqdm(total=epochs, desc="Training", unit="epoch")
    for _ in range(epochs):
        optimizer.zero_grad()
        loss = criterion(model(x),y.reshape(-1,1))
        loss.backward()
        optimizer.step()
        p_bar.set_postfix({"Loss": loss.item()})
        p_bar.update(1)
    p_bar.close()
    

In [4]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [16]:
x,y = load_coffee_data()
x = np.tile(x,(1000,1))
y = np.tile(y,(1000,1))
x = torch.from_numpy(x).cuda()
y = torch.from_numpy(y).cuda()

device(type='cuda', index=0)

In [17]:
model = CoffeeModel(x.shape[1]).double()
model.cuda()

CoffeeModel(
  (layer1): Linear(in_features=2, out_features=3, bias=True)
  (layer2): Linear(in_features=3, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [18]:
def normalize(x):
    mean = torch.mean(x,dim=0)
    std = torch.std(x,dim=0)
    return (x - mean) / std,mean,std

In [19]:
x_norm,mean,std = normalize(x)

In [20]:
x_norm.device

device(type='cuda', index=0)

In [21]:
train(x_norm,y,5000,model)

Training: 100%|██████████| 5000/5000 [00:56<00:00, 88.85epoch/s, Loss=0.00585]


In [24]:
X_test = np.array([
    [200,13.9],
    [200,17]]) 
X_test = torch.from_numpy(X_test).cuda()
X_test = (X_test - mean)/std 


In [25]:
pred = model(X_test)
pred

tensor([[9.7917e-01],
        [1.1511e-05]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SigmoidBackward0>)

In [26]:
for i in range(pred.shape[0]):
    if pred[i].item() >= 0.5:
        print("Good Coffee")
    else:
        print("Bad Coffee")

Good Coffee
Bad Coffee
